<a href="https://colab.research.google.com/github/tubagusalwasii/Generator-Makna-Lirik-Lagu-Drake/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pandas peft transformers accelerate bitsandbytes trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 24.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
import torch
import re
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, Trainer, default_data_collator
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from huggingface_hub import login
from google.colab import userdata

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)

In [ ]:
def clean_text(text):
    # Mengganti karakter encoding yang salah umum
    text = text.replace('â€”', '-')
    text = text.replace('â€™', "'")
    text = text.replace('â€œ', '"')
    text = text.replace('â€', '"')
    # Menghapus karakter '?' yang sering muncul di antara kata
    text = re.sub(r'(?<=\w)\?(?=\w)', "'", text)
    return text

In [ ]:
csv_path = "/content/drakesongs and meaning datasets.csv"
df = pd.read_csv(csv_path, encoding='latin-1', sep=';')

df.dropna(subset=["lyric_bait", "meaning_bait", "song_title", "album", "source"], inplace=True)
print("✅ Baris kosong berhasil dihapus!")

df.drop_duplicates(inplace=True)
print("✅ Baris duplikat berhasil dihapus!")

df['lyric_bait'] = df['lyric_bait'].apply(clean_text)
df['meaning_bait'] = df['meaning_bait'].apply(clean_text)
print("✅ Teks berhasil dibersihkan dari karakter aneh!")

df = df.sample(frac=1).reset_index(drop=True)
print("✅ Dataset berhasil diacak!")

✅ Baris kosong berhasil dihapus!
✅ Baris duplikat berhasil dihapus!
✅ Teks berhasil dibersihkan dari karakter aneh!
✅ Dataset berhasil diacak!


In [ ]:
jsonl_path = "/content/drake_meaning_dataset.jsonl"
with open(jsonl_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        item = {
            "lyrics": row["lyric_bait"],
            "meaning": row["meaning_bait"],
            "title": row["song_title"],
            "album": row["album"],
            "source": row["source"]
        }
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

dataset = load_dataset("json", data_files=jsonl_path, split="train")
print("✅ Dataset berhasil dikonversi ke format JSONL dan dimuat!")


Generating train split: 0 examples [00:00, ? examples/s]

✅ Dataset berhasil dikonversi ke format JSONL dan dimuat!


In [ ]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
base_model = prepare_model_for_kbit_training(base_model)
print("✅ Model dasar dan tokenizer berhasil dimuat!")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

✅ Model dasar dan tokenizer berhasil dimuat!


In [ ]:
lora_config = LoraConfig(
    r=32,                  # <-- REKOMENDASI: Ditingkatkan untuk kapasitas belajar
    lora_alpha=32,
    target_modules=[       # <-- REKOMENDASI: Targetkan lebih banyak modul
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ],
    lora_dropout=0.15,     # <-- REKOMENDASI: Ditingkatkan untuk mengurangi overfitting
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
print("✅ Model PEFT (LoRA) berhasil dikonfigurasi!")


✅ Model PEFT (LoRA) berhasil dikonfigurasi!


In [ ]:
def format_and_tokenize(example):
    full_text = f"Judul: {example['title']}\nAlbum: {example['album']}\nSumber: {example['source']}\n\nLirik: {example['lyrics']}\n\nMakna: {example['meaning']}"
    tokenized_inputs = tokenizer(
        full_text,
        truncation=True,
        max_length=1024,
        padding="max_length"
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"][:]
    return tokenized_inputs

tokenized_dataset = dataset.map(format_and_tokenize)
print("✅ Dataset berhasil diformat dan ditokenisasi!")


Map:   0%|          | 0/512 [00:00<?, ? examples/s]

✅ Dataset berhasil diformat dan ditokenisasi!


In [ ]:
training_args = TrainingArguments(
    output_dir="./drake-finetuned-model",   # <-- REKOMENDASI: Nama output baru
    num_train_epochs=3,                  # <-- REKOMENDASI: Dikurangi untuk mencegah overfitting
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_8bit",
    logging_steps=20,
    save_strategy="epoch",
    learning_rate=5e-5,                  # <-- REKOMENDASI: Sedikit diturunkan untuk stabilitas
    bf16=True,
    report_to=[],
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

print("🚀 Memulai proses fine-tuning...")
trainer.train()
print("✅ Proses fine-tuning selesai!")

/tmp/ipython-input-1537755666.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Memulai proses fine-tuning...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,2.115400
40,0.703300
60,0.601400
80,0.631500
100,0.636000
120,0.661100
140,0.503000
160,0.498100
180,0.601300
200,0.462800


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


✅ Proses fine-tuning selesai!


In [ ]:
output_dir = "/content/drake-finetuned-model-final" # Simpan ke folder baru
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model baru berhasil disimpan di: {output_dir}")

✅ Model baru berhasil disimpan di: /content/drake-finetuned-model-final


In [ ]:
!zip -r drake-finetuned-model-final.zip /content/drake-finetuned-model-final
print("✅ Model berhasil di-zip.")


  adding: content/drake-finetuned-model-final/ (stored 0%)
  adding: content/drake-finetuned-model-final/tokenizer.json (deflated 85%)
  adding: content/drake-finetuned-model-final/training_args.bin (deflated 53%)
  adding: content/drake-finetuned-model-final/adapter_model.safetensors (deflated 8%)
  adding: content/drake-finetuned-model-final/README.md (deflated 65%)
  adding: content/drake-finetuned-model-final/chat_template.jinja (deflated 71%)
  adding: content/drake-finetuned-model-final/special_tokens_map.json (deflated 63%)
  adding: content/drake-finetuned-model-final/adapter_config.json (deflated 56%)
  adding: content/drake-finetuned-model-final/tokenizer_config.json (deflated 96%)
✅ Model berhasil di-zip.
